In [1]:
import pandas as pd
import numpy as np
import re
import os
import torch
from ast import literal_eval
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, MultiLabelBinarizer, FunctionTransformer, StandardScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from gensim.models import Word2Vec
from transformers import DistilBertTokenizer, DistilBertModel
from concurrent.futures import ThreadPoolExecutor

/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/clean_cards.csv')
df.sample(5)

,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
22127,1,['W'],False,['Enchantment'],['none'],{X}: Testament of Faith becomes an X/X Wall cr...,['no keywords'],0,0,uncommon,False,2861,True,0.22
9762,6,['U'],False,['Creature'],['Serpent'],Islandwalk (This creature can't be blocked as ...,"['Landwalk', 'Islandwalk']",5,5,common,False,6799,True,0.13
21214,1,['R'],False,['Instant'],['none'],Street Spasm deals X damage to target creature...,['Overload'],0,0,uncommon,False,7275,False,0.08
13085,2,['R'],False,['Enchantment'],['Aura'],"Enchant creature\nEnchanted creature has ""{T}:...",['Enchant'],0,0,common,False,6729,True,0.03
23680,8,['R'],False,['Creature'],['Dragon'],"Flying\nWhenever a Dragon you control attacks,...",['Flying'],6,6,mythic,False,8668,False,11.97


#### Functions and models used for data processing

In [3]:
df['colors'] = df['colors'].apply(literal_eval)

In [4]:
mlb = MultiLabelBinarizer()
df= df.join(pd.DataFrame(mlb.fit_transform(df['colors']),
                                      columns=mlb.classes_,
                                      index=df.index))
legend_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['legendary']), columns=['legendary'])
             .add_prefix('_'))
booster_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['booster']), columns=['booster'])
             .add_prefix('_'))
reserved_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['reserved']), columns=['resrved'])
             .add_prefix('_'))

df = df.drop(['legendary', 'booster', 'reserved', 'colors'], axis=1)
df.head(3)

,cmc,types,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved
0,2,['Artifact'],['Equipment'],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,['Equip'],0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0
1,4,['Creature'],"['Bird', 'Rogue']",Flying\nWhen Aarakocra Sneak enters the battle...,['Flying'],1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0
2,5,['Creature'],"['Astartes', 'Warrior']",Trample\nMark of Chaos Ascendant — During your...,"['Mark of Chaos Ascendant', 'Trample']",5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0


##### Word 2 Vec Models

In [5]:
subtype_model = Word2Vec(sentences=df['sub_types'], vector_size=100, window=5, min_count=1, workers=4)
type_model = Word2Vec(sentences=df['types'], vector_size=100, window=5, min_count=1, workers=4)
keyword_model = Word2Vec(sentences=df['keywords'], vector_size=100, window=5, min_count=1, workers=4)

In [6]:
df['sub_types']=df['sub_types'].apply(lambda subtypes: np.sum([subtype_model.wv[word] for word in subtypes], axis=0))
df['types']=df['types'].apply(lambda types: np.sum([type_model.wv[word] for word in types], axis=0))
df['keywords']=df['keywords'].apply(lambda keywords: np.sum([keyword_model.wv[word] for word in keywords], axis=0))

#### Oracle text processing

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def get_embeddings(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].tolist()

In [8]:
df['text_embedding'] = df['oracle_text'].apply(get_embeddings)
df.drop(columns='oracle_text', inplace=True)

In [9]:
def reshape_array(arr):
    return arr.reshape(768,)

In [10]:
df['text_embedding']=df['text_embedding'].apply(np.array).apply(reshape_array)
embedding_columns = ['types', 'sub_types', 'keywords', 'text_embedding']
df['combined_embedding'] = df[embedding_columns].apply(np.concatenate, axis=1)
df.drop(columns=embedding_columns, inplace=True)
df.head(3)

/var/folders/tm/jmhp8xvj4_d4hh_01cf1jt_r0000gn/T/ipykernel_89224/478936331.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['combined_embedding'] = df[embedding_columns].apply(np.concatenate, axis=1)


,cmc,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved,combined_embedding
0,2,0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0,"[2.0790224075317383, -2.2230656147003174, 0.20..."
1,4,1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0,"[-3.4430713653564453, 0.8296054005622864, -2.1..."
2,5,5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0,"[-3.4430713653564453, 0.8296054005622864, -2.1..."


#### The Pipeline

In [11]:
X = df.drop(columns='usd')
y = df['usd']

In [12]:
numerical_features = ['power', 'toughness', 'released_at', 'cmc']
categorical_features = ['rarity']
remaining_features = X.drop(columns=['rarity', 'power', 'toughness', 'released_at', 'cmc'])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

pipeline = make_pipeline(preprocessor)

In [13]:
X_transformed = pipeline.fit_transform(X)

In [14]:
column_names = pipeline.get_feature_names_out()
column_names

array(['num__power', 'num__toughness', 'num__released_at', 'num__cmc',
       'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare',
       'cat__rarity_special', 'cat__rarity_uncommon', 'remainder__B',
       'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R',
       'remainder__U', 'remainder__W', 'remainder___legendary',
       'remainder___booster', 'remainder___resrved',
       'remainder__combined_embedding'], dtype=object)

In [15]:
processed_df = pd.DataFrame(X_transformed, columns=column_names)
processed_df.head()

,num__power,num__toughness,num__released_at,num__cmc,cat__rarity_common,cat__rarity_mythic,cat__rarity_rare,cat__rarity_special,cat__rarity_uncommon,remainder__B,remainder__C,remainder__G,remainder__N,remainder__R,remainder__U,remainder__W,remainder___legendary,remainder___booster,remainder___resrved,remainder__combined_embedding
0,-0.787117,-0.817482,0.832342,-0.743662,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,1,0,"[2.0790224075317383, -2.2230656147003174, 0.20..."
1,-0.237207,1.302307,0.931314,0.397975,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,1,0,"[-3.4430713653564453, 0.8296054005622864, -2.1..."
2,1.962433,1.832254,0.967891,0.968794,0.0,1.0,0.0,0.0,0.0,1,0,0,0,1,1,0,1,0,0,"[-3.4430713653564453, 0.8296054005622864, -2.1..."
3,-0.787117,-0.817482,-1.391463,-1.885299,1.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,1,0,"[2.369511127471924, -0.5469098091125488, 0.339..."
4,-0.787117,-0.817482,0.690338,-0.172843,0.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,"[2.0790224075317383, -2.2230656147003174, 0.20..."
